In [1]:
q = 487
m = 486
r = 496
n = 507
k = 165

## Key Generation

In [2]:
F.<z> = GF(q)
Flist = F.list()
Fstar = [i for i in F if i != 0]

def random_permutation_matrix(n):
    perm = Permutations(n).random_element()
    M = matrix(F, n, n)
    for i in range(n):
        M[i, perm(i+1)-1] = F(1)
    return M    

def homogeneous(n, m):
    M = zero_matrix(F, n, m)
    M[:m, :m] = random_permutation_matrix(m)
    M[m:, :n-m] = identity_matrix(F, n-m)
    return random_permutation_matrix(n) * M * random_permutation_matrix(m) 

def part_perm(r, n):
    M = zero_matrix(F, r, n)
    M[:, :r] = identity_matrix(F, r)
    return M * random_permutation_matrix(n)

def gen_error(n, t):
    res = zero_vector(F, n)
    for i in sample(range(n), t):
        res[i] = choice(Fstar)
    return res

In [3]:
alpha = sample(F.list(), m)
beta = [choice(Fstar) for i in range(m)]
C = codes.GeneralizedReedSolomonCode(alpha, k, beta)
S = random_matrix(F, k)
while S.rank() != k:
    S = random_matrix(F, k)
G = S*C.generator_matrix()

G1 = random_matrix(F, k, n)
while G1.rank() != k:
    G1 = random_matrix(F, k, n)
    
M1 = random_matrix(F, n)
M2 = random_matrix(F, n)
while M1.rank() != n:
    M1 = random_matrix(F, n)
while M2.rank() != n:
    M2 = random_matrix(F, n)

Q = part_perm(r, n)
R = homogeneous(n, m)
sol1 = (G1*M1*M2^-1).solve_right(G).T
sol2 = (G1*M1*M2^-1).right_kernel_matrix()
H2 = sol1 + random_matrix(F, sol1.nrows(), sol2.nrows()) * sol2
while H2.rank() != m: 
    sol2 = (G1*M^-1).right_kernel_matrix()
    H2 = sol1 + random_matrix(F, sol1.nrows(), sol2.nrows()) * sol2
P = (H2.T).solve_left(R)
G2 = (H2.T).left_kernel_matrix()
U = random_matrix(F, P.nrows(), G2.nrows())

Gpub = G1*M1
Epub = Q*(P+U*G2)*M2

In [4]:
# checking colunm weights in QR
QR = Q*R
c0 = 0
c1 = 0
c2 = 0
for i in (QR).columns():
    if i.hamming_weight() == 0:
        c0 += 1
    if i.hamming_weight() == 1:
        c1 += 1
    if i.hamming_weight() == 2:
        c2 += 1 
print(c0, c1, c2)

11 454 21


In [5]:
# Testing
msg = random_vector(F, k)
err = gen_error(r, (m-k) >> 1)
ciphertext = msg*Gpub + err * Epub
C.decode_to_message(ciphertext*M2^-1*(H2.T)) == msg * S

True

## Step 0: Finding auxilary matrices

In [6]:
# finding M3
Gpub_ = block_matrix([
    [identity_matrix(F, k), zero_matrix(F, k, n-k)]
])
M3 = Gpub.solve_right(Gpub_)
Gpub_kernel = Gpub.right_kernel_matrix().T
while M3.is_singular():
    M3 += Gpub_kernel * random_matrix(F, Gpub_kernel.ncols(), n)
    
Epub_ = Epub*M3

In [7]:
K = Epub_.right_kernel_matrix().T
K.rank(), K.rank() == n - r

(11, True)

In [8]:
sol1 = K[:k].solve_left(K[:k].rref()) 
sol2 = K[:k].left_kernel_matrix()
A = sol1 + random_matrix(F, k, sol2.nrows()) * sol2
while A.is_singular():
    A = sol1 + random_matrix(F, k, sol2.nrows()) * sol2

In [9]:
A_cal = A[K.rank():]

## Step 1: recovering the set of weight-1 columns of QR

An auxilary distingusher of non-GRS columns:

In [10]:
def square(G):
    N = G.nrows()
    res = zero_matrix(F, N*(N-1)//2 + N, G.ncols())
    r = 0
    for i in range(N):
        for j in range(i, N):
            res[r] = G[i].pairwise_product(G[j])
            r += 1
    return res

def remove_rand_columns_pass(G):
    if square(G).rank() == G.ncols():
        ns = 0
        B = codes.LinearCode(G).shortened([i for i in range(G.ncols() - ns, G.ncols())]).generator_matrix()
        while square(G).rank() >= G.ncols() - 2:
            ns += 1
            B = codes.LinearCode(G).shortened([i for i in range(G.ncols() - ns, G.ncols())]).generator_matrix()
        rr = square(B).rank()
        rnd_ind = []
        for i in range(0, G.ncols() - ns - 1):
            D = B.delete_columns([i])
            if square(D).rank() == rr - 1:
                rnd_ind.append(i)
        AA = G.delete_columns(rnd_ind)
        ns = 0
        B = codes.LinearCode(AA).shortened([i for i in range(ns)]).generator_matrix()
        while square(B).rank() >= B.ncols() - 2:
            ns += 1
            B = codes.LinearCode(AA).shortened([i for i in range(ns)]).generator_matrix()
        rr = square(B).rank()
        rnd_ind = []
        for i in range(ns, AA.ncols()):
            D = B.delete_columns([i-ns])
            if square(D).rank() == rr - 1:
                rnd_ind.append(i)
        return AA.delete_columns(rnd_ind)
    else:
        rr = square(G).rank()
        rnd_ind = []
        for i in range(G.ncols()):
            if square(G.delete_columns([i])).rank() == rr-1:
                rnd_ind.append(i)
        return G.delete_columns(rnd_ind)

def remove_rand_columns(G):
    AA = remove_rand_columns_pass(G)
    return remove_rand_columns_pass(AA)

def get_punctured_indices(original_matrix, punctured_matrix):
    original_cols = original_matrix.columns()
    punctured_cols = punctured_matrix.columns()
    punctured_indices = []
    for i, col in enumerate(original_cols):
        if col not in punctured_cols:
            punctured_indices.append(i)    
    return punctured_indices

In [11]:
# Finding L
L = Epub_.solve_right(identity_matrix(F, r))

# getting rid of $E_{pub}$'s kernel 
T = A_cal*L[:k]

# Getting rid of non-GRS columns in T
T_J = remove_rand_columns(T)

T.ncols(), T_J.ncols()

(496, 454)

In [12]:
J = get_punctured_indices(T, T_J)

W_J = block_matrix([
    [identity_matrix(F, r)[:, i] for i in range(r) if i not in J]
])

# Let's check number of common columns of QR_ and ground truth QR
def common_columns(A, B):
    common_count = 0
    for col_A in A.columns():
        for col_B in B.columns():
            if col_A == col_B:
                common_count += 1
                break
    return common_count

# checking
W_J.ncols(), c1, common_columns(W_J, QR)

(454, 454, 454)

## Step 2: recovering weight-2 columns of QR

In [13]:
def zero_rows(G):
    zero_indices = [i for i in range(G.nrows()) if G.row(i).is_zero()]
    return zero_indices

# def checkGRS(G):
#     G2 = remove_rand_columns_pass(G)
#     if G == G2:
#         return True
#     return False

# slightly more efficient version of the previous function
def checkGRS(G):
    if square(G).rank() == G.ncols():
        ns = 0
        G1 = codes.LinearCode(G).shortened([i for i in range(ns)]).generator_matrix()
        while square(G1).rank() >= G1.ncols() - 2:
            ns += 1
            G1 = codes.LinearCode(G).shortened([i for i in range(ns)]).generator_matrix()
        rr = square(G1).rank()
        D = G1[:, :-1]
        if square(D).rank() == rr - 1:
            return False
    else:
        rr = square(G).rank()
        if square(G[:, :-1]).rank() == rr-1:
            return False
    return True

def find_new_column(B):
    zero_row_inds = zero_rows(B)
    for i in range(len(zero_row_inds)):
        for j in range(i+1, len(zero_row_inds)):
            new_col = zero_vector(F, r)
            new_col[zero_row_inds[i]] = 1
            new_col[zero_row_inds[j]] = 1
            B_test = B.augment(new_col)
            T = A_cal * (Epub_.solve_right(B_test)[:k])
            if checkGRS(T):
                return B_test
    return B

In [14]:
B = copy(W_J)
while B.ncols() != m:
    old_B = copy(B)
    B = find_new_column(B)
    if B == old_B:
        break
    print("+")

+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+


In [15]:
# checking
B.ncols(), c1 + c2, common_columns(B, QR)

(475, 475, 475)

## Step 3: finding a partial support of $\widetilde{C}$ 

In [16]:
Y = Epub_.solve_right(B)

G3 = A_cal * Y[:k,:]

In [17]:
def recover_GRS_support(G, x0, x1, xk):
    n = G.ncols()
    k = G.nrows()
    G1 = G.rref()
    x = [q+1]*n
    x[0], x[1], x[k] = x0, x1, xk
    count = 3
    i = 0; i_ = 1; j_ = k
    for j in range(k+1, n):
        gamma = (G1[i,j] * G1[i_, j_]) / (G1[i, j_] * G1[i_, j])
        denom = (x[j_] - x[i]) - gamma*(x[j_] - x[i_])
        if denom != 0:
            xj = (x[i_]*(x[j_]-x[i]) - gamma*x[i]*(x[j_] - x[i_])) / denom
            if xj in x:
                return x, False
            x[j] = xj
        else:
            return x, False
    i_ = 0; j = k; j_ = k+1
    for i in range(2, k):
        gamma = (G1[i,j] * G1[i_, j_]) / (G1[i, j_] * G1[i_, j])
        denom = gamma*(x[j_] - x[i_]) - (x[j] - x[i_])
        if denom != 0:
            xi = (gamma*x[j]*(x[j_]-x[i_]) - (x[j] - x[i_])*x[j_]) / denom
            if xi in x:
                return x, False
            x[i] = xi
        else:
            return x, False            
    return x, True

In [18]:
candidate_supports = []

G3_sq = codes.LinearCode(square(G3)).generator_matrix().rref()
x0, x1 = 0, 1
for xk in F.list():
    if (xk == 0) or (xk == 1):
        continue
    fl = False
    new_x, fl = recover_GRS_support(G3_sq, x0, x1, xk)
    if fl:
        candidate_supports.append(new_x)
     
print(len(candidate_supports), candidate_supports[0])

13 [0, 1, 329, 319, 443, 307, 6, 214, 297, 91, 144, 135, 257, 465, 479, 450, 460, 433, 434, 155, 281, 101, 185, 351, 429, 380, 77, 261, 56, 406, 377, 14, 7, 245, 191, 44, 48, 357, 360, 457, 46, 369, 266, 367, 201, 453, 59, 441, 336, 62, 51, 393, 13, 389, 260, 310, 203, 250, 148, 436, 325, 430, 11, 321, 173, 445, 63, 175, 87, 94, 220, 259, 311, 341, 184, 472, 90, 386, 395, 33, 70, 40, 467, 174, 350, 477, 84, 100, 391, 12, 335, 83, 138, 440, 193, 242, 41, 130, 476, 227, 249, 145, 108, 169, 73, 353, 292, 69, 26, 196, 420, 317, 207, 439, 299, 421, 423, 313, 343, 36, 211, 198, 247, 279, 372, 358, 171, 361, 352, 244, 356, 43, 296, 354, 272, 412, 23, 275, 147, 401, 442, 468, 373, 215, 150, 435, 27, 234, 107, 119, 344, 411, 387, 323, 45, 384, 419, 314, 223, 300, 375, 194, 235, 190, 398, 24, 22, 409, 105, 269, 464, 212, 80, 182, 304, 289, 328, 158, 459, 221, 222, 285, 89, 312, 408, 228, 362, 253, 388, 327, 57, 79, 295, 32, 85, 164, 106, 15, 428, 219, 333, 383, 131, 181, 66, 72, 146, 116, 60, 47

## Step 4: completing the support and recovering the full secret key

In [19]:
YK = Y.augment(K)

G4 = YK[:k,:]

In [20]:
# some stuff to solve matrix equation in SageMath
def fold(v, m_, rk, l):
    M = zero_matrix(F, m_ + rk, m_ + l)
    M[0:m_, 0:m_] = diagonal_matrix(v[:m_])
    M[m_:] = matrix(F, rk, m_ + l, v[m_:])
    return M

In [21]:
alpha_ = candidate_supports[0]

In [22]:
afull = copy(alpha_)
# case when there's no zero columns in QR
if len(alpha_) == m:
    l = 0
    num_of_unknowns = (c1 + c2) + K.ncols()*(c1+c2+l)
    I_nou = identity_matrix(F, num_of_unknowns)
    RS_pc_mat = codes.GeneralizedReedSolomonCode(alpha_, k).parity_check_matrix()
    tmp = matrix([(G4 * fold(i, c1 + c2, K.ncols(), l) * RS_pc_mat.T).list() for i in I_nou])
    sol = tmp.left_kernel()
    X = fold(sol.random_element(), c1 + c2, K.ncols(), l)
    if (G4*X).rank() != m:
        X = fold(sol.random_element(), c1 + c2, K.ncols(), l)

# general case
import itertools
l = m-c1-c2
fl = False
for Gamma in map(list, itertools.combinations([i for i in F if i not in alpha_], l)):
    num_of_unknowns = (c1 + c2) + K.ncols()*(c1+c2+l)
    I_nou = identity_matrix(F, num_of_unknowns)
    RS_pc_mat = codes.GeneralizedReedSolomonCode(alpha_ + Gamma, k).parity_check_matrix()
    tmp = matrix([(G4 * fold(i, c1 + c2, K.ncols(), l) * RS_pc_mat.T).list() for i in I_nou])
    sol = tmp.left_kernel()
    if sol.dimension() == 0:
        continue
    for j in range(10):
        a_ = sol.random_element()
        if (0 not in a_[:c1+c2]) and (rank(G4*fold(a_, c1 + c2, K.ncols(), l)) == k):
            X = fold(a_, c1 + c2, K.ncols(), l)
            afull = alpha_ + Gamma
            fl = True
            print("+")
            break
    if fl:
        break    
    print("-")

+


In [23]:
G5 = G4*X
C_rec = codes.GeneralizedReedSolomonCode(afull, k)
S_rec = C_rec.generator_matrix().solve_left(G5)

ciphertext_ = ciphertext * M3 * YK * X
msg_rec = C_rec.decode_to_message(ciphertext_) * S_rec^-1

msg_rec, msg

((50, 284, 320, 234, 475, 180, 389, 146, 421, 65, 143, 219, 311, 375, 40, 467, 155, 264, 299, 108, 479, 356, 278, 46, 272, 29, 470, 7, 481, 2, 282, 331, 367, 271, 163, 370, 404, 461, 164, 423, 442, 144, 168, 412, 370, 473, 374, 246, 310, 29, 107, 36, 93, 460, 199, 471, 152, 210, 218, 473, 208, 316, 93, 280, 249, 34, 234, 172, 366, 127, 155, 347, 363, 377, 441, 188, 52, 421, 472, 387, 82, 130, 266, 46, 437, 417, 253, 148, 218, 220, 251, 317, 314, 98, 428, 274, 299, 328, 25, 451, 156, 440, 238, 90, 372, 187, 294, 369, 112, 240, 242, 239, 246, 385, 272, 374, 20, 183, 167, 408, 238, 160, 159, 82, 111, 194, 163, 406, 61, 205, 228, 68, 191, 421, 427, 453, 27, 432, 391, 199, 351, 290, 24, 286, 461, 409, 483, 408, 54, 45, 166, 193, 415, 280, 453, 414, 425, 474, 97, 360, 87, 143, 281, 339, 229),
 (50, 284, 320, 234, 475, 180, 389, 146, 421, 65, 143, 219, 311, 375, 40, 467, 155, 264, 299, 108, 479, 356, 278, 46, 272, 29, 470, 7, 481, 2, 282, 331, 367, 271, 163, 370, 404, 461, 164, 423, 442, 144,